In [1]:
from Header import *
%matplotlib inline

In [2]:
df = pd.read_excel("sentences_with_sentiment.xlsx")

In [5]:
df.head()

,ID,Sentence,Positive,Negative,Neutral
0,1,The results in 2nd line treatment show an ORR ...,1,0,0
1,2,The long duration of response and high durable...,1,0,0
2,3,The median OS time in the updated results exce...,0,0,1
3,4,"Therefore, the clinical benefit in 2nd line tr...",1,0,0
4,5,"The data provided in 1st line, although prelim...",1,0,0


In [6]:
df.describe()

,ID,Positive,Negative,Neutral
count,266.000000,266.000000,266.000000,266.000000
mean,133.500000,0.601504,0.135338,0.263158
std,76.931788,0.490511,0.342729,0.441177
min,1.000000,0.000000,0.000000,0.000000
25%,67.250000,0.000000,0.000000,0.000000
50%,133.500000,1.000000,0.000000,0.000000
75%,199.750000,1.000000,0.000000,1.000000
max,266.000000,1.000000,1.000000,1.000000


In [11]:
df[['Positive', 'Negative', 'Neutral']].sum(axis=0)

Positive    160
Negative     36
Neutral      70
dtype: int64

Unbalanced data, especially small number of negative values.<br>
In general small amount of data, I guess not enough for Deep Neural Networks.
Classifier with small number of features might work.

In [12]:
df[['Positive', 'Negative', 'Neutral']].sum(axis=1).max()

1

No sentence has more than one category.

In [41]:
for i,g in df.groupby('Sentence'):
    if len(g) > 1:
        print("More than one sentences are the same.")
        break

More than one sentences are the same.


In [36]:
df_clean = df.drop_duplicates(subset=['Sentence','Positive', 'Negative', 'Neutral'])

In [42]:
print("Positive sentences:")
for s in df_clean[df_clean.Positive==1].Sentence:
    print(s)

Positive sentences:
The results in 2nd line treatment show an ORR of 33% with some patients having durable responses resulting in an apparent PFS plateau
The long duration of response and high durable response rate observed after 6-month minimum follow-up is further substantiated by the 12-months minimum follow-up analysis
Therefore, the clinical benefit in 2nd line treatment is considered clinically meaningful and the magnitude of the effect is significant compared to chemotherapy
The data provided in 1st line, although preliminary, showed activity of avelumab in mMCC in terms of response rate, and evidence of a similar durable response from the small subset of patients with a longer follow-up compared to 2nd line treatment
Taking into account the intrinsic limitation of single arm studies, the rarity of the disease and the challenges to compare the results with data from historical controls and in the literature, the currently available data are deemed to support the efficacy of avel

In [17]:
print("Negative sentences:")
for s in df[df.Negative==1].Sentence:
    print(s)
Same sentence multiple times

Negative sentences:
This means that further evidence on this medicinal product is awaited
The CHMP considers the following measures necessary to address the missing efficacy data for 1st line treatment in the context of a conditional MA:
• PASS: German real-world cohort study should be submitted as additional PhV activity to address the missing information of safety and efficacy in immune compromised patients
The CHMP considers the following measures necessary to address issues related to safety: 
• PASS: German real-world cohort study should be submitted as additional PhV activity to address the missing information of safety and efficacy in immune compromised patients
Although dataset of AFL patients has been updated, data are still considered very limited to reach firm conclusion about safety proflie.
Additional safety data from maintenance study period CT-P10 3.3 and follow-up period should be provided (see RMP).
As questions still remain on the long term safety, ongoing open-label 

<p>
Key Task:<br>
feasability of an automated sentiment analy- sis tool for EPARs, which
Identifies optimal and suboptimal clinical trial properties.
</p>
<p>
Mentioned questions:
    <ul>
    <li> Is the dataset balanced? No </li>
    <li> Is the amount of data sufficient for allowing a hold-out dataset?
        Probalby no, only 29 points in train (in average). But k-fold cross-validation should be used. </li>
        <li>
            Do you have enough data to consider deep neural architectures or might good feature engineering with more shallow models suffice?
            Probably not, check numbers.
            </li>
        <li> During the data collection process, for some sentences multiple experts disagreed on the sentiment of a given sentence, how could you capture such an ambiguity in your model and potentially notify users about such unclear instances?
        Give chances, not only categories.
            Maybe, train more than one model. And compare results in the end.
        </li>
        <li>
        How does your model come to a specific conclusion, what about model interpretability?
        </li>
        <li>
        Think beyond the pure sentiment analysis.
        </li>
</p>
<p>
Additional Questions:
two or three categories?
</p>